In [1]:
import requests

url = 'http://www.pitchfork.com/robots.txt'
response  = requests.get(url)
print(response.text)

User-agent: *
Allow: /
Disallow: /search/
Disallow: /search

Sitemap: https://pitchfork.com/sitemap.xml
Sitemap: https://pitchfork.com/branded-sitemap.xml
Sitemap: https://pitchfork.com/feed/google-latest-news/sitemap-google-news
Sitemap: https://pitchfork.com/feed/sitemap


In [57]:
test = requests.get("https://pitchfork.com/reviews/albums/?page=2")
print(test)

<Response [200]>


In [78]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup

## allows us to use reg expressions to search fields
import re

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC

import random
import time
import csv
import os

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

# user agent
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')


Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2656.18 Safari/537.36


### II. Selenium to scrape album contents

In [100]:
# -*- coding: utf-8 -*-
"""
Created on Sun Dec  3 20:13:09 2017
@author: Evan
"""
import requests 
from bs4 import BeautifulSoup
import pandas as pd

# driver = webdriver.Chrome(chromedriver,chrome_options=options)

# #_url = "https://www.discogs.com/sell/list?sort=price%2Cdesc&limit=100&year1=1900&year2=1970&format=Vinyl&price=over40&genre=Jazz&currency=USD&style=Hard+Bop&page="
# # not just bebop
# _url = "https://pitchfork.com/reviews/albums/?genre=jazz/"
# url= _url + str(pagestart) 

# # starting url
# driver.get(url)


'''
This function parses the HTML of the page and attempts to gather attributes like
artist name, album, genre, date, and the review text itself, instead inputting a
null value if the requested element is not found on the page. All of the data are
put into a Pandas dataframe and returned for use in the gather function.
VARIABLES
album_link - A string that refers to the album section of a link to a Pitchfork 
review.
e.g. '/reviews/albums/neil-young-promise-of-the-real-visitor/'
'''
def gather_info(album_link):
    #page = requests.get("https://pitchfork.com" + album_link) #request URL
    page = requests.get("https://pitchfork.com" + album_link) #request URL
    soup = BeautifulSoup(page.content, 'html.parser') #parse with beautifulsoup
    title = str(soup.find('title').string) #album and artist 
    try:
        score = float((soup.find(class_="score").string)) #score
    except AttributeError:
        score = None
    try:
        genre = soup.find(class_="genre-list__link").string #genre
    except AttributeError:
        genre = None
    sents = [element.text for element in soup.find_all('p')] #cleaned text output
    string = " ".join(sents)
    try:
        date = str(soup.find(class_="pub-date").string) #date
    except AttributeError:
        date = None
    #create dataframe with column labels
    d = {'artist': [get_artist(title)], 'album': [get_album(title)], 'score': [score], 'genre': [genre], 'review': [string], 'best': [1 if "Best new" in string else 0], 'date': [date]}
    df = pd.DataFrame(data=d)
    return df


In [101]:
'''
This function starts at Pitchfork's album reviews page and searches through a
requested number of pages from a given start page, adding each album link to the
queue to be scraped by the gather_info function and returning them in a list.
VARIABLES
pages, startPage - Integers that refer to the number of pages to scrape and the
page to start on, respectively, while scraping through Pitchfork's album reviews
page.
. 
'''
def gather_links(pages, startPage):
    pageList = [] #list of album review pages
    linkList = [] #list of album links
    
    for x in range(startPage,(startPage+pages)): #check the first n pages after the requested one
        #pageList.append(requests.get("https://pitchfork.com/reviews/albums/?genre=jazz&page=" + str(x))) #add each page to list
        pageList.append(requests.get("https://pitchfork.com/reviews/albums/?genre=jazz&page=" + str(x))) #add each page to list
        print('x',x,'pagelist',pageList)
    for page in pageList:
        print('page',page)
        soup = BeautifulSoup(page.content, 'html.parser') #parse its contents
        #print(soup)
        links = soup.find_all(class_="review__link") #gather its links (in raw html)
        print('links',links)
        for link in links: #for each link
            linkList.append(link.get('href')) #append only the link itself
        
    return linkList


In [102]:
'''
This function retreives the artist name from the scraped title string.
VARIABLES
title - A string of a cleaned Pitchfork album review title.
'''
def get_artist(title):
    str = ''
    for character in title: #for each character in title
        #add to string up until ':' 
        #if character is not ':':
        if character != ':':
            str += character
        else:
            break
    return str


In [103]:
'''
This function retreives the album name from the scraped title string.
VARIABLES
title - A string of a cleaned Pitchfork album review title.
'''          
def get_album(title):
    str = ''
    #find ':' and index and start there
    index = title.find(":")
    title = title[index+2:]
    #for each character afterwards, add it until '|'
    for character in title:
        #if character is '|':
        if character == '|':
            break
        else:
            str +=character
    return str[:-14] #return just the title



In [104]:
'''
This function uses the other two to gather the requested number of pages starting
from a given page, then adding them all into a single Pandas dataframe, which
is then exported to a CSV file in the given location of the user's desktop.
VARIABLES
pages, startPages - Integers that refer to the number of pages to scrape and the
page to start on, respectively, while scraping through Pitchfork's album reviews
page.
fileLocation - A string that gives a path in the user's desktop where the data
should be saved. 
e.g. 'C:/Users/Evan/Documents'
fileName - A string that gives the desired name of the .csv file. 
e.g. 'p4kReview'
'''
def gather(pages, startPage, fileLocation, fileName):
    linkList = gather_links(pages, startPage) #gather links
    print(linkList)
    first = True #special variable for first scrape
    newDF = pd.DataFrame()
#     global data
#     data=0
    for link in linkList: #for each link        
        
        data = gather_info(link) #gather info
        #if first, newDF becomes the initial dataframe
        if first:
            newDF = data
            first = False
        #otherwise append it
        else:
            newDF = newDF.append(data, ignore_index = True)
    #when scraping complete, export to .csv 
    newDF.to_csv(path_or_buf = fileLocation + "/" + fileName + ".csv")
    #return true if gather was successful
    return True

In [105]:
gather(52,1,'/Users/chrischan/Documents/02_Class/Metis/bootcamp/githubrepo/Metis_proj5/data','pitchfork_jazz')

x 1 pagelist [<Response [200]>]
x 2 pagelist [<Response [200]>, <Response [200]>]
x 3 pagelist [<Response [200]>, <Response [200]>, <Response [200]>]
x 4 pagelist [<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
x 5 pagelist [<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
x 6 pagelist [<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
x 7 pagelist [<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
x 8 pagelist [<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
x 9 pagelist [<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
x 10 pagelist [<Response [200]>, <Response [200]>, <Response [20

x 30 pagelist [<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
x 31 pagelist [<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>

x 43 pagelist [<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
x 44 pagelist [<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>

links [<a class="review__link" href="/reviews/albums/femi-kuti-made-kuti-legacy/"><div class="review__artwork artwork"><div class=""><img alt="" src="https://media.pitchfork.com/photos/602ed922e06a43d9ea0fe986/1:1/w_160/Femi%20Kuti%20&amp;%20Made%20Kuti%20-%20Legacy+.jpg"/></div></div><div class="review__title"><ul class="artist-list review__title-artist"><li>Femi Kuti</li><li>Made Kuti</li></ul><h2 class="review__title-album">Legacy +</h2></div></a>, <a class="review__link" href="/reviews/albums/archie-shepp-jason-moran-let-my-people-go/"><div class="review__artwork artwork"><div class=""><img alt="" src="https://media.pitchfork.com/photos/6026aecd8bff13f31ec23444/1:1/w_160/Archie%20Shepp%20&amp;%20Jason%20Moran%20-%20Let%20My%20People%20Go.jpg"/></div></div><div class="review__title"><ul class="artist-list review__title-artist"><li>Archie Shepp</li><li>Jason Moran</li></ul><h2 class="review__title-album">Let My People Go</h2></div></a>, <a class="review__link" href="/reviews/albums/p

links [<a class="review__link" href="/reviews/albums/jyoti-mama-you-can-bet/"><div class="review__artwork artwork"><div class="review__artwork--with-notch"><img alt="Image may contain: Human, Person, Advertisement, Poster, Hair, Brochure, Paper, and Flyer" src="https://media.pitchfork.com/photos/5f4e5f8224167c1baabf781d/1:1/w_160/Mama,%20You%20Can%20Bet!_jyoti.jpg"/></div></div><div class="review__title"><ul class="artist-list review__title-artist"><li>Jyoti</li></ul><h2 class="review__title-album">Mama, You Can Bet!</h2></div></a>, <a class="review__link" href="/reviews/albums/nubya-garcia-source/"><div class="review__artwork artwork"><div class="review__artwork--with-notch"><img alt="Image may contain: Modern Art, Art, Skin, and Painting" src="https://media.pitchfork.com/photos/5f43d022f6eb11025fede8d4/1:1/w_160/source_nubya%20garcia.jpg"/></div></div><div class="review__title"><ul class="artist-list review__title-artist"><li>Nubya Garcia</li></ul><h2 class="review__title-album">Sour

...

links [<a class="review__link" href="/reviews/albums/1148-peace-with-every-step/"><div class="review__artwork artwork"><div class=""><img alt="" src="https://media.pitchfork.com/photos/5929a47a13d1975652138d01/1:1/w_160/ed2ac16a.gif"/></div></div><div class="review__title"><ul class="artist-list review__title-artist"><li>Build an Ark</li></ul><h2 class="review__title-album">Peace With Every Step</h2></div></a>, <a class="review__link" href="/reviews/albums/6933-the-floating-glass-key-in-the-sky/"><div class="review__artwork artwork"><div class=""><img alt="Image may contain: Text, Alphabet, Number, and Symbol" src="https://media.pitchfork.com/photos/5929a310c0084474cd0c01be/1:1/w_160/3baf0652.jpg"/></div></div><div class="review__title"><ul class="artist-list review__title-artist"><li>Rainstick Orchestra</li></ul><h2 class="review__title-album">The Floating Glass Key in the Sky</h2></div></a>, <a class="review__link" href="/reviews/albums/11836-chiaroscuro-humcrush/"><div class="review

links [<a class="review__link" href="/reviews/albums/5879-dodeka/"><div class="review__artwork artwork"><div class=""><img alt="" src="https://media.pitchfork.com/photos/5929b95713d197565213ae24/1:1/w_160/623655ac.jpg"/></div></div><div class="review__title"><ul class="artist-list review__title-artist"><li>Arne Nordheim</li></ul><h2 class="review__title-album">Dodeka</h2></div></a>, <a class="review__link" href="/reviews/albums/966-never-too-late-but-always-too-early/"><div class="review__artwork artwork"><div class=""><img alt="" src="https://media.pitchfork.com/photos/5929a3b9c0084474cd0c0326/1:1/w_160/78152867.gif"/></div></div><div class="review__title"><ul class="artist-list review__title-artist"><li>Peter Brötzmann</li><li>William Parker</li><li>Hamid Drake</li></ul><h2 class="review__title-album">Never Too Late But Always Too Early</h2></div></a>, <a class="review__link" href="/reviews/albums/8234-observing-systems/"><div class="review__artwork artwork"><div class=""><img alt="I

links [<a class="review__link" href="/reviews/albums/5395-change-is-coming/"><div class="review__artwork artwork"><div class=""><img alt="Image may contain: Paper, and Money" src="https://media.pitchfork.com/photos/5929c39bc0084474cd0c34e7/1:1/w_160/7cb96623.jpg"/></div></div><div class="review__title"><ul class="artist-list review__title-artist"><li>Money Mark</li></ul><h2 class="review__title-album">Change Is Coming</h2></div></a>, <a class="review__link" href="/reviews/albums/6172-doing-their-own-thing/"><div class="review__artwork artwork"><div class=""><img alt="" src="https://media.pitchfork.com/photos/59299ff69d034d5c69bf25d3/1:1/w_160/a357cb12.gif"/></div></div><div class="review__title"><ul class="artist-list review__title-artist"><li>Maceo Parker</li></ul><h2 class="review__title-album">Doing Their Own Thing</h2></div></a>, <a class="review__link" href="/reviews/albums/5098-run-come-save-me/"><div class="review__artwork artwork"><div class=""><img alt="Image may contain: Huma

links [<a class="review__link" href="/reviews/albums/5424-fragments-of-freedom/"><div class="review__artwork artwork"><div class=""><img alt="" src="https://media.pitchfork.com/photos/5929c3b8eb335119a49ed799/1:1/w_160/bdff28ea.gif"/></div></div><div class="review__title"><ul class="artist-list review__title-artist"><li>Morcheeba</li></ul><h2 class="review__title-album">Fragments of Freedom</h2></div></a>, <a class="review__link" href="/reviews/albums/7439-disappeared/"><div class="review__artwork artwork"><div class=""><img alt="" src="https://media.pitchfork.com/photos/5929a52613d1975652138e9f/1:1/w_160/af0836b3.gif"/></div></div><div class="review__title"><ul class="artist-list review__title-artist"><li>Spring Heel Jack</li></ul><h2 class="review__title-album">Disappeared</h2></div></a>, <a class="review__link" href="/reviews/albums/8004-the-mirror-conspiracy/"><div class="review__artwork artwork"><div class=""><img alt="Image may contain: Face, Human, Person, and Head" src="https:/

True